# Universidad Autónoma de Aguascalientes
# Departamento: Ciencias de la Computación
# Materia: Machine y Deep Learning
# Profesor: Dr. Francisco Javier Luna Rosas
###   Alumnos:
### - Haniel Joab Cruz Chávez
### - Boris Iván Bernal Orozco
### - Abraham Padilla Pizaña
### - Juan Andres Rodriguez Parra
### - Isbaal Antonio Medina Montañez
### - Raul Iván Rivera Pérez
### - Francisco Javier Altamira Mata
### Semestre: Enero-Junio 2025

# Examen_1 AI_ICI_Feb_2025. Reconocimiento de Patrones (Firmas en Cheques Off-Line)


### Objetivo: Extraer firmas de cheques y clasificarlas en base a la firma de un individuo. Para ello, se realizarán las siguientes tareas:
### 1. Aumentar la saturación de las imágenes de los cheques.
### 2. Dividir las imágenes de los cheques en 4 partes.
### 3. Guardar la esquina inferior derecha de las imágenes de los cheques.
### 4. Obtener la firma de la esquina inferior derecha de las imágenes de los cheques.
### 5. Binarizar la imagen de la firma.
### 6. Extraer características de la firma.
### 7. Clasificar las firmas en base a la firma de un individuo.

## Paso 1: Importar librerías necesarias

In [121]:
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import random

## Paso 2: Cargar las imágenes de los cheques

In [122]:
for i in range(1, 106):
    # Cargar la imagen original
    imagen = Image.open(
        "E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Cheques/{}.jpg".format(
            i))

    # Aumentar la saturación
    enhancer = ImageEnhance.Color(imagen)
    imagen_saturada = enhancer.enhance(4.0)

    # Obtiene las dimensiones de la imagen
    ancho, alto = imagen_saturada.size

    # Divide la imagen en 4 partes
    mitad_ancho = ancho // 2
    mitad_alto = alto // 2
    parte_inferior_derecha = imagen_saturada.crop((mitad_ancho, mitad_alto, ancho, alto))
    esquina_inferior_derecha = parte_inferior_derecha

    # Guarda la esquina inferior derecha
    esquina_inferior_derecha.save(
        "E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Esquinas_Cheques/{}.jpg".format(
            i))
    # Cierra la imagen original
    imagen.close()

## Paso 3: Obtener la firma de la esquina inferior derecha de las imágenes de los cheques

In [123]:
for i in range(1, 106):
    # Cargar la imagen recién guardada
    image = cv2.imread(
        'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Esquinas_Cheques/{}.jpg'.format(
            i))

    # Definir un rango de color azul
    lower_blue = np.array([100, 0, 0])  # Valores más bajos para el azul muy oscuro
    upper_blue = np.array([255, 50, 50])  # Valores más altos para el azul muy oscuro

    # Crear una máscara para el rango de colores azules
    mask = cv2.inRange(image, lower_blue, upper_blue)

    # Aplicar la máscara para eliminar el fondo
    result = cv2.bitwise_and(image, image, mask=mask)

    # Guardar la imagen sin fondo
    cv2.imwrite(
        'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Extraidas/{}.jpg'.format(
            i), result)

## Paso 4: Binarizar la imagen de la firma

In [124]:
for i in range(1, 106):
    # Cargar la imagen
    image = cv2.imread(
        'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Extraidas/{}.jpg'.format(
            i))

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar un umbral a la imagen y convertirla en una imagen binaria
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Guardar la imagen binaria
    cv2.imwrite(
        'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Binarizadas/{}.jpg'.format(
            i), thresh)

## Paso 5: Cargar las imágenes binarizadas de las firmas

In [125]:
# Inicializar la lista con None para los 105 elementos
firmas = [None] * 105

# Cargar las imágenes en escala de grises
for i in range(0, 105):
    img_path = 'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Binarizadas/{}.jpg'.format(i+1)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Advertencia: No se pudo cargar la imagen {i} desde {img_path}")
    else:
        firmas[i] = img

## Paso 6: Crear los elementos estructurales para la extracción de características

In [126]:
# Crear elementos estructurales
EE1 = np.array([[0, 0, 1, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 1, 0, 0]], dtype=np.uint8)

EE2 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE3 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]], dtype=np.uint8)

EE4 = np.array([[1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0]], dtype=np.uint8)

EE5 = np.array([[0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0]], dtype=np.uint8)

EE6 = np.array([[0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0]], dtype=np.uint8)

EE7 = np.array([[1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE8 = np.array([[0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE9 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE10 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1]], dtype=np.uint8)

EE11 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0],
                   [1, 0, 0, 0, 0]], dtype=np.uint8)

EE12 = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1]], dtype=np.uint8)

EE13 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0]], dtype=np.uint8)

EE14 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 1, 0, 0]], dtype=np.uint8)
EE15 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0]], dtype=np.uint8)

EE16 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 1, 1, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE17 = np.array([[0, 0, 0, 0, 1],
                   [0, 1, 1, 1, 0],
                   [1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE18 = np.array([[0, 0, 0, 1, 1],
                   [1, 1, 1, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE19 = np.array([[1, 1, 1, 0, 0],
                   [0, 0, 0, 1, 1],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE20 = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 1, 1, 0],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE21 = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 1, 0],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE22 = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 0, 1, 0]], dtype=np.uint8)

EE23 = np.array([[1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0]], dtype=np.uint8)

EE24 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1],
                   [0, 1, 1, 1, 0],
                   [1, 0, 0, 0, 0]], dtype=np.uint8)

EE25 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1],
                   [1, 1, 1, 1, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE26 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 1],
                   [0, 1, 1, 1, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE27 = np.array([[0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0]], dtype=np.uint8)

EE28 = np.array([[0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0]], dtype=np.uint8)

EE29 = np.array([[0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0]], dtype=np.uint8)

EE30 = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0]], dtype=np.uint8)

EE31 = np.array([[0, 0, 1, 0, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0]], dtype=np.uint8)

EE32 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 1, 0, 1, 0],
                   [1, 0, 1, 0, 1]], dtype=np.uint8)

EE33 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 0, 1, 0, 1],
                   [0, 1, 0, 1, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE34 = np.array([[0, 0, 0, 0, 0],
                   [0, 1, 0, 1, 0],
                   [1, 0, 1, 0, 1],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE35 = np.array([[1, 0, 1, 0, 1],
                   [0, 1, 0, 1, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE36 = np.array([[0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 1, 1, 1, 0],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE37 = np.array([[0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 1, 1, 0, 0],
                   [0, 0, 0, 1, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE38 = np.array([[0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 1, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE39 = np.array([[0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 1, 0, 0, 0],
                   [0, 0, 1, 1, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE40 = np.array([[1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1],
                   [0, 1, 1, 1, 0]], dtype=np.uint8)

EE41 = np.array([[0, 1, 1, 1, 0],
                   [1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE42 = np.array([[0, 1, 1, 1, 1],
                   [1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 1, 1, 1, 1]], dtype=np.uint8)

EE43 = np.array([[1, 1, 1, 1, 0],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1],
                   [1, 1, 1, 1, 0]], dtype=np.uint8)

EE44 = np.array([[1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1],
                   [1, 1, 1, 1, 1],
                   [1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1]], dtype=np.uint8)

EE45 = np.array([[0, 1, 1, 1, 0],
                   [1, 0, 0, 0, 1],
                   [1, 1, 1, 1, 1],
                   [1, 0, 0, 0, 1],
                   [1, 0, 0, 0, 1]], dtype=np.uint8)

EE46 = np.array([[1, 1, 1, 1, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0],
                   [1, 1, 1, 1, 1]], dtype=np.uint8)

EE47 = np.array([[1, 0, 0, 0, 1],
                   [0, 1, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 1, 0],
                   [1, 0, 0, 0, 1]], dtype=np.uint8)

EE48 = np.array([[0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1]], dtype=np.uint8)

EE49 = np.array([[0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1]], dtype=np.uint8)

EE50 = np.array([[0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0]], dtype=np.uint8)

EE51 = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0],
                   [0, 1, 0, 0, 0]], dtype=np.uint8)

EE52 = np.array([[1, 1, 1, 1, 1],
                   [1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1],
                   [1, 1, 1, 1, 1]], dtype=np.uint8)

EE53 = np.array([[0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0]], dtype=np.uint8)

EE54 = np.array([[0, 1, 0, 1, 0],
                   [0, 1, 0, 1, 0],
                   [0, 1, 0, 1, 0],
                   [0, 1, 0, 1, 0],
                   [0, 1, 0, 1, 0]], dtype=np.uint8)

elementosEstructurados = [EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,EE11,EE12,EE13,EE14,EE15,EE16,EE17,EE18,EE19,EE20,EE21,EE22,EE23,EE24,EE25,EE26,EE27,EE28,EE29,EE30,EE31,EE32,EE33,EE34,EE35,EE36,EE37,EE38,EE39,EE40,EE41,EE42,EE43,EE44,EE45,EE46,EE47,EE48,EE49,EE50,EE51,EE52,EE53,EE54]
elementosEstructuradosNombre = ["EE1", "EE2", "EE3", "EE4", "EE5", "EE6", "EE7", "EE8", "EE9", "EE10", "EE11", "EE12", "EE13", "EE14", "EE15", "EE16", "EE17", "EE18", "EE19", "EE20", "EE21", "EE22", "EE23", "EE24", "EE25", "EE26", "EE27", "EE28", "EE29", "EE30", "EE31", "EE32", "EE33", "EE34", "EE35", "EE36", "EE37", "EE38", "EE39", "EE40", "EE41", "EE42", "EE43", "EE44", "EE45", "EE46", "EE47", "EE48", "EE49", "EE50", "EE51", "EE52", "EE53", "EE54"]
elementoObtenido = np.zeros((len(firmas),len(elementosEstructurados)))

## Paso 7: Extraer características de las firmas

In [127]:
for i in range(len(firmas)):
    if firmas[i] is None:
        continue  # Saltar si la imagen no se cargó correctamente
    for j in range(len(elementosEstructurados)):
        # Aplicar dilatación
        img_dilatada = cv2.erode(firmas[i], elementosEstructurados[j], iterations=1)
        img_erosion = cv2.dilate(img_dilatada, elementosEstructurados[j], iterations=1)

        suma = np.sum(img_erosion == 0)
        elementoObtenido[i][j] = suma

## Paso 8: Escribir las características en un archivo CSV, agregar la columna "Verdadera" y crear firmas sintéticas positivas y negativas

In [128]:
ruta_csv = 'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/ElementosEstructuralesEnFirmas.csv'

# Agregar la columna "Verdadera" en el encabezado
elementosEstructuradosNombre.append("Verdadera")
elementoObtenido = np.array(elementoObtenido)
todas_las_firmas = []

# Se le asigna "SI" a las 105 firmas originales en la columna "Verdadera"
for fila in elementoObtenido:
    todas_las_firmas.append(fila.tolist() + ["SI"])

# Crear las 50 firmas sintéticas verdaderas
firmas_sinteticas_v = []
for i in range(50):
    inicio = i  # Desplazamiento en la ventana
    fin = inicio + 105  # Se toma del inicio al inicio + 105
    nueva_firma = np.trunc(np.mean(elementoObtenido[inicio:fin], axis=0) - np.std(elementoObtenido[inicio:fin], axis=0))
    todas_las_firmas.append(nueva_firma.tolist() + ["SI"])

# Crear las 155 firmas sintéticas falsas con valores aleatorios entre 1 y 300
for _ in range(155):
    firma_falsa = [random.randint(1, 300) for _ in range(elementoObtenido.shape[1])]
    todas_las_firmas.append(firma_falsa + ["NO"])

# Guardar en CSV
with open(ruta_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(elementosEstructuradosNombre)
    writer.writerows(todas_las_firmas)

print("Archivo CSV generado exitosamente.")

Archivo CSV generado exitosamente.


## Paso 9: Cargar 30% de los datos y 2 cheques falsos para el testing

In [129]:
imgFalsa1 = cv2.imread('E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Binarizadas/106.jpg', cv2.IMREAD_GRAYSCALE)
imgFalsa2 = cv2.imread('E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Binarizadas/107.jpg', cv2.IMREAD_GRAYSCALE)

# Inicializar la lista con None para los 105 elementos
firmas_prediccion = [None] * 21

# Cargar las imágenes en escala de grises
for i in range(0, 21):
    img_path = 'E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/Firmas/Firmas_Binarizadas/{}.jpg'.format(i+5)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Advertencia: No se pudo cargar la imagen {i} desde {img_path}")
    else:
        firmas_prediccion[i] = img

## Paso 10: Cargamos el dataset de los elementos estructurales en firmas

In [130]:
df = pd.read_csv('E:/Universidad/6to Semestre/Aprendizaje Inteligente/1er Parcial/FO-121500-10 Examen_1 AI_ICI_Feb_2025/ElementosEstructuralesEnFirmas.csv')
df.head()

,EE1,EE2,EE3,EE4,EE5,EE6,EE7,EE8,EE9,EE10,...,EE46,EE47,EE48,EE49,EE50,EE51,EE52,EE53,EE54,Verdadera
0,2099.0,2108.0,2599.0,2599.0,2349.0,2349.0,3108.0,2608.0,2608.0,3108.0,...,2282.0,2000.0,1894.0,1864.0,1717.0,1707.0,2328.0,2173.0,2201.0,SI
1,2080.0,2144.0,2580.0,2580.0,2330.0,2330.0,3144.0,2644.0,2644.0,3144.0,...,2325.0,1973.0,1874.0,1904.0,1776.0,1715.0,2330.0,2158.0,2187.0,SI
2,2351.0,2359.0,2851.0,2851.0,2601.0,2601.0,3359.0,2859.0,2859.0,3359.0,...,2534.0,2282.0,2101.0,2152.0,1998.0,2075.0,2580.0,2399.0,2455.0,SI
3,2027.0,2082.0,2527.0,2527.0,2277.0,2277.0,3082.0,2582.0,2582.0,3082.0,...,2363.0,2039.0,1881.0,1917.0,1682.0,1710.0,2369.0,2139.0,2174.0,SI
4,2242.0,2255.0,2742.0,2742.0,2492.0,2492.0,3255.0,2755.0,2755.0,3255.0,...,2414.0,2137.0,2013.0,1992.0,1875.0,1870.0,2446.0,2298.0,2351.0,SI


## Paso 11: Elimina la variable catégorica, deja las variables predictorias en X


In [131]:
X = df.iloc[:,:54]
print(X.head())

      EE1     EE2     EE3     EE4     EE5     EE6     EE7     EE8     EE9  \
0  2099.0  2108.0  2599.0  2599.0  2349.0  2349.0  3108.0  2608.0  2608.0   
1  2080.0  2144.0  2580.0  2580.0  2330.0  2330.0  3144.0  2644.0  2644.0   
2  2351.0  2359.0  2851.0  2851.0  2601.0  2601.0  3359.0  2859.0  2859.0   
3  2027.0  2082.0  2527.0  2527.0  2277.0  2277.0  3082.0  2582.0  2582.0   
4  2242.0  2255.0  2742.0  2742.0  2492.0  2492.0  3255.0  2755.0  2755.0   

     EE10  ...    EE45    EE46    EE47    EE48    EE49    EE50    EE51  \
0  3108.0  ...  2163.0  2282.0  2000.0  1894.0  1864.0  1717.0  1707.0   
1  3144.0  ...  2168.0  2325.0  1973.0  1874.0  1904.0  1776.0  1715.0   
2  3359.0  ...  2440.0  2534.0  2282.0  2101.0  2152.0  1998.0  2075.0   
3  3082.0  ...  2174.0  2363.0  2039.0  1881.0  1917.0  1682.0  1710.0   
4  3255.0  ...  2343.0  2414.0  2137.0  2013.0  1992.0  1875.0  1870.0   

     EE52    EE53    EE54  
0  2328.0  2173.0  2201.0  
1  2330.0  2158.0  2187.0  
2  2580.

## Paso 12: Dejar la variable a predecir Y


In [132]:
y = df.iloc[:,54:55]
print(y.head())

  Verdadera
0        SI
1        SI
2        SI
3        SI
4        SI


## Paso 13: Dividir el dataset en 70% para entrenamiento y 30% para testing

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Paso 14: Mediante el constructor inicializa la instancia_red


In [134]:
instancia_red = MLPClassifier()
print(instancia_red)

MLPClassifier()


## Paso 15: Entrena el modelo llamado al metodo fit

In [135]:
instancia_red.fit(X_train.values, y_train.iloc[:, 0].values)

MLPClassifier()

## Paso 16: Imprime las predicciones en testing


In [136]:
print("Las predicciones en Testing son: {}".format(instancia_red.predict(X_test.values)))

Las predicciones en Testing son: ['NO' 'SI' 'NO' 'NO' 'SI' 'NO' 'SI' 'SI' 'NO' 'NO' 'SI' 'SI' 'NO' 'NO'
 'SI' 'SI' 'NO' 'SI' 'NO' 'SI' 'SI' 'NO' 'SI' 'SI' 'NO' 'SI' 'SI' 'NO'
 'NO' 'SI' 'SI' 'SI' 'NO' 'SI' 'SI' 'SI' 'SI' 'NO' 'SI' 'NO' 'SI' 'SI'
 'NO' 'SI' 'NO' 'SI' 'NO' 'SI' 'SI' 'SI' 'NO' 'SI' 'NO' 'NO' 'NO' 'SI'
 'SI' 'NO' 'SI' 'SI' 'NO' 'SI' 'SI' 'SI' 'NO' 'SI' 'SI' 'NO' 'NO' 'SI'
 'NO' 'NO' 'NO' 'NO' 'NO' 'NO' 'SI' 'NO' 'NO' 'SI' 'NO' 'SI' 'SI' 'SI'
 'SI' 'NO' 'NO' 'NO' 'SI' 'SI' 'NO' 'NO' 'SI']


## Paso 17: Función para calcular los índices de calidad de la precicción


In [137]:
def indices_general(MC, nombres = None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    precision_categoria = pd.DataFrame(MC.diagonal()/np.sum(MC, axis = 1)).T
    if nombres != None:
        precision_categoria.columns = nombres
    return {"Matriz de Confusión": MC,
            "Precisión Global": precision_global,
            "Error Global": error_global,
            "Precisión por Categoría": precision_categoria}

## Paso 18: Índices de Calidad del Modelo


In [138]:
prediccion = instancia_red.predict(X_test.values)
MC = confusion_matrix(y_test, prediccion)
indices = indices_general(MC, list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusión:
[[43  2]
 [ 0 48]]

Precisión Global:
0.978494623655914

Error Global:
0.021505376344086002

Precisión por Categoría:
         NO   SI
0  0.955556  1.0


## Paso 19: Obtenemos las características de las firmas falsas y verdaderas

In [139]:
firmas_prediccion = firmas_prediccion + [imgFalsa1, imgFalsa2]

# Ingresamos 2 firmas verdaderas y 2 falsas para que el modelo las clasifique
for i in range(len(firmas_prediccion)):
    if firmas_prediccion[i] is None:
        continue  # Saltar si la imagen no se cargó correctamente
    for j in range(len(elementosEstructurados)):
        # Aplicar dilatación
        img_dilatada = cv2.erode(firmas_prediccion[i], elementosEstructurados[j], iterations=1)
        img_erosion = cv2.dilate(img_dilatada, elementosEstructurados[j], iterations=1)

        suma = np.sum(img_erosion == 0)
        elementoObtenido[i][j] = suma

## Paso 20: Realizamos la predicción de las firmas falsas y verdaderas

In [140]:
prediccion_firma = instancia_red.predict(elementoObtenido)

print("Predicción de firma verdadera: {}".format(prediccion_firma[0]))
print("Predicción de firma verdadera: {}".format(prediccion_firma[1]))
print("Predicción de firma falsa: {}".format(prediccion_firma[21]))
print("Predicción de firma falsa: {}".format(prediccion_firma[22]))

Predicción de firma verdadera: SI
Predicción de firma verdadera: SI
Predicción de firma falsa: NO
Predicción de firma falsa: NO


## Conclusiones

#### En este examen extrajimos características de firmas en cheques con la finalidad de crear un modelo de clasificación usando NNBP (Redes Neuronales de Propagación hacia Atrás) que nos permitiera identificar si una firma es verdadera o falsa. Para ello aprendimos a dividir los cheques en 4 partes y tomar la parte inferior derecha, parte en la cual se encontraban las firmas, posteriormente nos dimos cuenta de que estábamos teniendo problemas para extraer la firma para lo cual investigamos métodos de preprocesamiento de imágenes y descubrimos que subiendo la saturación obtendríamos mejores resultados ya que de esta manera se resaltaría aún más el color azul de la firma y con esto logramos evitar tomar otros pixeles no relacionados, aprendimos a aplicar una máscara para extraer las firmas y binarizamos la imagen para poder extraer las características de las firmas. Creamos elementos estructurales para la extracción de características, aplicamos dilatación y después erosión en cada una de las firmas con el fin de obtener unas firmas más nítidas y con ello obtener sus características. Extrajimos las características de las firmas haciendo uso de los elementos estructurales y se escribieron en un archivo CSV, se agregó la columna "Verdadera" y se crearon firmas sintéticas positivas y negativas, todo este proceso nos ayudó a crear un dataset preciso el cual se usó para entrenar el modelo de clasificación.

## Referencias

### [1] MLPClassifier. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
### [2] OpenCV: Image Filtering. (s. f.). https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga3a8d0f29a9d616e9e6b07cd99c226dea
### [3] Preprocesamiento con visión artificial Parte VII: Operaciones morfológicas. (2021, 16 febrero). ICHI.PRO. https://ichi.pro/es/preprocesamiento-con-vision-artificial-parte-viioperaciones-morfologicas-222672754581160
### [4] Sosa-Costa, A. (2023, 25 julio). Transformaciones Morfológicas - ▷ Cursos de Programación de 0 a Experto © Garantizados. ▷ Cursos de Programación de 0 A Experto © Garantizados. https://unipython.com/transformaciones-morfologicas/
